# JOINS 

This jupyter is to join the datasets with their 
one to many notes files.

In [196]:
import string
import bz2
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import nltk.corpus  
import time
import math
import re
import spacy
from functools import reduce
from nltk.text import Text
from nltk.corpus import stopwords
from itertools import combinations
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from textblob import TextBlob
from nltk.stem.wordnet import WordNetLemmatizer
# from keras.preprocessing import text, sequence
# from keras.utils import to_categorical
# from keras.layers import Bidirectional, GlobalMaxPool1D,GlobalMaxPool2D, GlobalMaxPool3D,Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation
# from keras.models import Model, Sequential
# from keras.callbacks import EarlyStopping, ModelCheckpoint
%matplotlib inline
plt.style.use('fivethirtyeight')


In [8]:
os.getcwd()

'C:\\Users\\t.cri.kcheung\\capstone_nlp\\src\\joins'

In [14]:
# get all files in subfolder
mypath = "Z:/FINAL OUTPUT v2/"
onlyfiles = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f))]

In [325]:
onlyfiles

['Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv']

# Modular

In [326]:
onlyfiles_sorted = sorted(onlyfiles, key=lambda x:(x!="PATIENT_COHORT", x))

In [347]:
listtmp = list_

In [348]:
for i in range(0,len(listtmp[0:1])):
    print(i)
    tmp1 = pd.DataFrame(listtmp[i].iloc[:,0])
#     print(tmp1)
    listtmp[i] = listtmp[i].iloc[:,1:].add_suffix(suffix[i])
    listtmp[i] = pd.concat((tmp1,(listtmp[i])),axis=1)


0


In [349]:
listtmp[0].head()

,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient
0,32124,90.0,F,White,Not Hispanic or Latino,NaN
1,74912,63.0,F,Black/African-American,Not Hispanic or Latino,NaN
2,81545,65.0,M,White,Not Hispanic or Latino,2015-06-21 00:00:00.000
3,93110,47.0,F,Black/African-American,Not Hispanic or Latino,2013-04-07 00:00:00.000
4,118647,52.0,F,Black/African-American,Not Hispanic or Latino,NaN


In [335]:
tmp1.head()

,patient_id
0,32124
1,74912
2,81545
3,93110
4,118647


In [344]:
pd.concat((tmp1,(listtmp[0])),axis=1).head()

,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient
0,32124,90.0,F,White,Not Hispanic or Latino,NaN
1,74912,63.0,F,Black/African-American,Not Hispanic or Latino,NaN
2,81545,65.0,M,White,Not Hispanic or Latino,2015-06-21 00:00:00.000
3,93110,47.0,F,Black/African-American,Not Hispanic or Latino,2013-04-07 00:00:00.000
4,118647,52.0,F,Black/African-American,Not Hispanic or Latino,NaN


In [380]:
pd.merge(list_[0],list_[5],on="patient_id",how="left")

,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id_radiology,order_proc_id_deid_radiology,ordering_date_off_radiology,exam_type_radiology
0,32124,90.0,F,White,Not Hispanic or Latino,NaN,NaN,NaN,NaN,NaN
1,74912,63.0,F,Black/African-American,Not Hispanic or Latino,NaN,NaN,NaN,NaN,NaN
2,81545,65.0,M,White,Not Hispanic or Latino,2015-06-21 00:00:00.000,NaN,NaN,NaN,NaN
3,93110,47.0,F,Black/African-American,Not Hispanic or Latino,2013-04-07 00:00:00.000,NaN,NaN,NaN,NaN
4,118647,52.0,F,Black/African-American,Not Hispanic or Latino,NaN,NaN,NaN,NaN,NaN
5,122487,43.0,M,Black/African-American,Not Hispanic or Latino,NaN,NaN,NaN,NaN,NaN
6,122522,74.0,M,Black/African-American,Not Hispanic or Latino,2014-04-28 00:00:00.000,NaN,NaN,NaN,NaN
7,136921,57.0,F,Black/African-American,Not Hispanic or Latino,NaN,NaN,NaN,NaN,NaN
8,137154,54.0,F,Black/African-American,Not Hispanic or Latino,NaN,NaN,NaN,NaN,NaN
9,140791,50.0,M,Black/African-American,Not Hispanic or Latino,NaN,NaN,NaN,NaN,NaN


In [370]:
frame = pd.DataFrame()
list_ = []

def readin(listoffiles,like):
    for f in listoffiles:
        if f.find(like) == -1:
            print("skipped:",f)
        else:
            print("processed",f)
            df = pd.read_csv(f,index_col=None, header=0,sep="|")
            df.columns = [x.lower() for x in df.columns]
            list_.append(df)
    suffix = ('_patient', '_encounter', '_encounter_provider', '_encounter_social','_dx_info','_radiology')
    for i in range(0,len(list_)):
        print(i)
        tmp1 = pd.DataFrame(list_[i].iloc[:,0])
    #     print(tmp1)
        list_[i] = list_[i].iloc[:,1:].add_suffix(suffix[i])
        list_[i] = pd.concat((tmp1,(list_[i])),axis=1)
    frame = reduce(lambda x, y: pd.merge(x, y, on = 'patient_id',how="left"), list_)
    return(frame)

In [371]:
cohort1_dta = readin(onlyfiles_sorted,"COHORT_1")

processed Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv
skipped: Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv
processed Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv
skipped: Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv
processed Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv
skipped: Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv
processed Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv
skipped: Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv
processed Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv
skipped: Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv
processed Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv
skipped: Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv
0
1
2
3
4
5


In [372]:
cohort1_dta.shape

(83877, 119)

In [369]:
cohort1_dta.iloc[80000,0:]

patient_id                                        1973766
age_at_last_encounter                                  75
gender                                                  F
race                                                White
ethnicity                          Not Hispanic or Latino
date_of_death_d                                       NaN
encounter_id_x                                    5111136
enc_eio                                                 O
visit_status_1                                  Completed
visit_status_2                                   Canceled
visit_status_3                                        NaN
visit_status_4                                        NaN
adm_date_d                        2013-04-30 00:00:00.000
disc_date_d                       2013-04-30 00:00:00.000
start_date_d                      2013-04-30 00:00:00.000
end_date_d                        2013-04-30 00:00:00.000
admit_source_1                                        NaN
admit_source_2

In [178]:
cohort1_dta.columns

Index(['abstinence_yn', 'adm_date_d', 'admit_source_1', 'admit_source_2',
       'admit_source_3', 'admitting_prov_id', 'admitting_provider_department',
       'age_at_last_encounter', 'alcohol_oz_per_wk', 'alcohol_src_c',
       ...
       'tobacco_pak_per_dy', 'tobacco_used_years', 'tobacco_user',
       'unknown_fam_hx_yn', 'visit_status_1', 'visit_status_2',
       'visit_status_3', 'visit_status_4', 'years_education', 'zip_d'],
      dtype='object', length=116)

### Prior to loading in the second cohort let's randomly sample in 10k rows


In [237]:
cohort2samp = pd.read_csv(onlyfiles_sorted[1],sep="|")
cohort2samp.head()

,PATIENT_ID,AGE_AT_LAST_ENCOUNTER,gender,race,ETHNICITY,DATE_OF_DEATH_D
0,49,79.0,M,White,Not Hispanic or Latino,NaN
1,99,86.0,F,White,Not Hispanic or Latino,NaN
2,104,84.0,M,White,Not Hispanic or Latino,NaN
3,114,66.0,F,White,Not Hispanic or Latino,NaN
4,118,92.0,F,Black/African-American,Not Hispanic or Latino,NaN


In [242]:
cohort2sampdf = cohort2samp.sample(n=10000,random_state=10,replace=False)
cohort2sampdf.shape

(10000, 6)

In [244]:
cohort2sampdf.to_csv(join(mypath,"01_PATIENT_COHORT_2_NEW.csv"),sep="|")

### Replace _list with the new filename

In [247]:
onlyfiles_sorted[1] = join(mypath,"01_PATIENT_COHORT_2_NEW.csv")
onlyfiles_sorted[1]

'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv'

In [373]:
onlyfiles_sorted[0:4]

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv']

In [252]:
cohort2_dta = readin(onlyfiles_sorted[0:4],"COHORT_2")

skipped: Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv
processed Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv
skipped: Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv
processed Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2850: DtypeWarning: Columns (6,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


KeyboardInterrupt: 

In [180]:
cohort2_dta.shape

(5695268, 335)

In [181]:
cohort2_dta.head()

,abstinence_yn,adm_date_d,admit_source_1,admit_source_2,admit_source_3,admitting_prov_id,admitting_provider_department,age_at_last_encounter,alcohol_oz_per_wk,alcohol_src_c,...,tobacco_pak_per_dy,tobacco_used_years,tobacco_user,unknown_fam_hx_yn,visit_status_1,visit_status_2,visit_status_3,visit_status_4,years_education,zip_d
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Write csvs

In [182]:
cohort1_dta.to_csv(join(mypath,"merged_cohort1.csv"),sep="|")
cohort2_dta.to_csv(join(mypath,"merged_cohort2.csv"),sep="|")

# Joining to rad notes  

Note the links will be messy until we get clarification.

Second string is "encounter_id"

In [259]:
# extract encounter_id from notes
mypathnotes = "Z:/FINAL OUTPUT v2/NOTES/"
onlyfilesnotes = [join(mypathnotes,f) for f in listdir(mypathnotes) if isfile(join(mypathnotes, f))]
onlyfilesnotes_sorted = sorted(onlyfilesnotes, key=lambda x:(x!="cohort", x))

# cohort 1
notes1 = pd.read_csv(onlyfilesnotes_sorted[0],index_col=None, header=0,sep=",")
notes1.columns = [x.lower() for x in notes1.columns]

# cohort 2
# notes2 = pd.read_csv(onlyfilesnotes_sorted[0],index_col=None, header=0,sep="|")
# notes1.columns = [x.lower() for x in df.columns]

In [260]:
notes1.head()

,unnamed: 0,scantype,scandate,clininfo,technique,comparison,findings,impressions,elecsig,patient,rawtext
0,0,"XR CHEST PA/LATERAL,",2015-06-22**]> 8:24 AM,**NAME<HHH> **AGE<in 50s> years old. Reason: s...,XR CHEST PA/LATERAL,**DATE<[**2015-06-14**]> and previous.,Status post right lung biopsies. No significan...,No significant abnormality following multiple ...,**DATE<[**2015-06-22**]> 10:06 AM **NAME<III J...,5253312_70864302,"XR CHEST PA/LATERAL, **DATE<[**2015-06-22**]> ..."
1,1,CT CHEST W,2015-02-06**]> 12:25 PM,**AGE<in 60s>-year-old male with eye on the,Contrast enhanced chest CT. 120 mL of Omnipaq...,Outside CXR dated **DATE<[**2015-01-11**]>,LUNGS AND PLEURA: Mild subpleural reticulation...,Mild subpleural interstitial fibrosis of undet...,**DATE<[**2015-02-06**]> 2:26 PM **NAME<CCC DD...,5049178_68355312,CT CHEST W **DATE<[**2015-02-06**]> 12:25 PM\...
2,2,"XR CHEST PA/LATERAL,",2015-05-23**]> 10:08 AM,**NAME<FFF> **AGE<in 60s> years old. One month...,XR CHEST PA/LATERAL,**DATE<[**2015-05-02**]>.,Elevation of the left hemidiaphragm unchanged....,Severe interstitial abnormality without visibl...,**DATE<[**2015-05-23**]> 10:42 AM [**Doctor Fi...,5861184_69548263,"XR CHEST PA/LATERAL, **DATE<[**2015-05-23**]> ..."
3,3,"XR CHEST PA/LATERAL,",2015-06-24**]> 12:40 PM,"**NAME<DDD> **AGE<in 50s> years old. Dyspnea, ...",XR CHEST PA/LATERAL,**DATE<[**2014-09-03**]> and previous,Diffuse fine reticular opacities throughout bo...,Diffuse groundglass opacities consistent with ...,**DATE<[**2015-06-24**]> 2:22 PM **NAME<EEE FF...,5630079_69208276,"XR CHEST PA/LATERAL, **DATE<[**2015-06-24**]> ..."
4,4,These outside images were loaded for compariso...,NaN,NaN,NaN,NaN,NaN,NaN,**DATE<[**2018-04-03**]> 8:07 AM Canned Report,8425699_110461983,These outside images were loaded for compariso...


In [264]:
# split 1
notes1[['patient1','patient2']] = notes1['patient'].str.split('_',expand=True)

# split2
# notes2[['patient1','patient2']] = notes2['patient'].str.split('_',expand=True)

### Now join notes for cohort1

In [ ]:
cohort1_dta_notes = pd.merge(cohort1_dta, notes1, on = 'patient_id',how="left")

In [375]:
cohort1_dta[['patient_id','encounter_id_radiology']]

,patient_id,encounter_id_radiology
0,32124,NaN
1,32124,NaN
2,32124,NaN
3,32124,NaN
4,74912,NaN
5,74912,NaN
6,74912,NaN
7,74912,NaN
8,74912,NaN
9,74912,NaN


In [376]:
cohort1_dta[cohort1_dta.encounter_id_radiology.notnull()]

,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id_encounter,enc_eio_encounter,visit_status_1_encounter,visit_status_2_encounter,...,dx_12_dx_info,dx_desc_12_dx_info,dx12_rank_dx_info,dx_13_dx_info,dx_desc_13_dx_info,dx13_rank_dx_info,encounter_id_radiology,order_proc_id_deid_radiology,ordering_date_off_radiology,exam_type_radiology
